# TIME SERIES MODEL FOR COTY VOLUME FORECAST

In [1]:
%matplotlib inline
import pandas as pd

In [2]:
data = pd.read_csv("Data/coty_dataset.csv",sep = ';', header=0,index_col=0)
data.head()

In [3]:
to_remove = (['year','week','clean_su','ship_ma','ship_ma30'])

In [4]:
df = data[data.columns.difference(to_remove)]

In [5]:
df.head()

In [6]:
df.index = pd.to_datetime(df.index)

In [7]:
df.head()

In [8]:
import plotly

In [9]:
plotly.tools.set_credentials_file(username='mamadou.baro', api_key='HCz7EG3n0600vrKJpjuk')

In [10]:
import plotly.plotly as ply
import cufflinks as cf
df.iplot(title="Ship units Evolution Mar 2015 -- Feb 2019", theme='pearl')

In [11]:
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df.ship_units.values, freq=30, model= 'multiplicative')
fig = result.plot()
plot_mpl(fig)

'https://plot.ly/~mamadou.baro/6'

In [12]:
from pyramid.arima import auto_arima 

/ds_data/anaconda3/lib/python3.6/site-packages/pyramid/__init__.py:68: UserWarning:


    The 'pyramid' package will be migrating to a new namespace beginning in 
    version 1.0.0: 'pmdarima'. This is due to a package name collision with the
    Pyramid web framework. For more information, see Issue #34:
    
        https://github.com/tgsmith61591/pyramid/issues/34
        
    The package will subsequently be installable via the name 'pmdarima'; the
    only functional change to the user will be the import name. All imports
    from 'pyramid' will change to 'pmdarima'.
    



In [19]:
import numpy as np
df = df[np.isfinite(df['ship_units'])]

In [20]:
stepwise_model = auto_arima(df, start_p=1, start_q=1,
                           max_p=10, max_q=10, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=10951.153, BIC=10975.554, Fit time=9.334 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=12094.483, BIC=12104.244, Fit time=0.124 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=11608.075, BIC=11627.596, Fit time=1.883 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=10969.944, BIC=10989.465, Fit time=3.960 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=10954.310, BIC=10983.592, Fit time=7.446 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=11542.658, BIC=11562.179, Fit time=1.002 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=10954.179, BIC=10983.461, Fit time=25.161 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=10956.384, BIC=10990.547, Fit time=25.286 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=10945.362, BIC=10974.644, Fit time=7.377 seconds
Fit ARIM

In [21]:
stepwise_model.aic()

10945.361833868257

# TRAIN TEST SPLIT

In [23]:
df.head()

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 986 entries, 2015-03-25 to 2019-02-26
Data columns (total 1 columns):
ship_units    986 non-null int64
dtypes: int64(1)
memory usage: 15.4 KB


In [68]:
train = df.loc['2015-03-25':'2018-12-26']

In [69]:
train.tail()

In [70]:
test = df.loc['2018-12-26':]

In [59]:
test.head()

In [60]:
test.tail()

In [71]:
len(test)

40

In [72]:
future_forecast = stepwise_model.predict(n_periods=40)

In [73]:
future_forecast

array([214.92155879, 202.72310642, 215.73111175, 215.37262932,
       218.83632055, 217.35622191, 222.0455854 , 224.94047162,
       211.16604803, 222.62273409, 209.17197969, 212.06383021,
       219.89366864, 210.14601141, 218.47014381, 217.76701283,
       220.72162714, 219.14434438, 223.77204302, 226.65080635,
       212.86954094, 224.32509541, 210.87485442, 213.76799497,
       221.59938534, 211.85339046, 220.17922525, 219.47781271,
       222.43415148, 220.85859554, 225.48802189, 228.36851328,
       214.58897607, 226.04625878, 212.59774606, 215.49261489,
       223.32573353, 213.58146693, 221.90903   , 221.20934574])

In [74]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['Prediction'])

In [75]:
future_forecast.head()

In [66]:
test.head()

In [76]:
pd.concat([test,future_forecast],axis=1).iplot()